# Visualize the results of the ATA analysis

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import panel as pn
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')
from bokeh.resources import INLINE
import paths

import functions_bondjango as bd
import functions_kinematic as fk
import functions_plotting as fp
import functions_misc as fm
import functions_data_handling as fd
import pandas as pd
import numpy as np
import h5py
# define the name to be used for the saved figures
save_name = 'actTAs'
line_width = 5
# define the interval length of the TA
interval_length = 5

In [ ]:
from bokeh.themes.theme import Theme

theme = Theme(
    json={
    'attrs' : {
        'Figure' : {
            'background_fill_color': '#FFFFFF',
            'border_fill_color': '#FFFFFF',
            'outline_line_color': '#FFFFFF',
        },
#         'Grid': {
#             'grid_line_dash': [6, 4],
#             'grid_line_alpha': .3,
#         },
        'Text':
            {
                'text_font': 'Arial',
            },

        'Axis': {
            'major_label_text_color': 'black',
            'axis_label_text_color': 'black',
            'major_tick_line_color': 'black',
            'minor_tick_line_color': 'black',
            'axis_line_color': "black"
        }
    }
})

hv.renderer('bokeh').theme = theme

In [ ]:
# get the data paths
try: 
    data_path = snakemake.input[0]
except NameError:
    # define the search string
    search_string = 'result:succ, lighting:normal, rig:miniscope, imaging:doric, analysis_type:trigAveCA'
    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_path = data_all[0]['analysis_path']
print(data_path)

# get the suffix for the filename
suffix = fp.search2path(search_string)
# load the data
# with pd.HDFStore(data_all[0]['analysis_path']) as h:
with pd.HDFStore(data_path) as h:

    # get a list of the existing keys
    keys = h.keys()
    # if it's only one key, just load the file
    if len(keys) == 1:
        data = h[keys[0]]
    else:   
        # allocate a dictionary for them
        data = {}
        # extract the animals present
        animal_list = [el.split('/')[1] for el in keys]
        # get the unique animals
        unique_animals = np.unique(animal_list)
        # for all the animals
        for animal in unique_animals:
            # allocate a dictionary for the tables
            sub_dict = {}
            # get the unique dates for this animal
            date_list = [el.split('/')[2] for el in keys if animal in el]
            # for all the unique dates
            for date in date_list:
                # get the corresponding key
                current_key = [el.split('/')[4] for el in keys if (animal in el) and (date in el)]
                # allocate a dictionary
                sub2_dict = {}
                # for all the keys
                for key in current_key:
                    sub2_dict[key] = h['/'+'/'.join((animal,date,'trigAveCA',key))]
                # load the table into the dictionary (minus the d at the beginning, added cause natural python naming)
                sub_dict[date[1:]] = sub2_dict
            # save the dictionary into the corresponding entry of the animal dictionary
            data[animal] = sub_dict
            
# # print(data)

# # Get the distribution of the pair of target parameters

# # define the search string
# # search_string = 'result:succ, lighting:normal, rig:miniscope, =analysis_type:aggEnc'
# search_string_enc = search_string.replace('trigAveCA', 'aggFullCA')
# # query the database for data to plot
# data_all = bd.query_database('analyzed_data', search_string_enc)
# data_path = data_all[0]['analysis_path']
# print(data_path)

# # load the data
# data_full = fd.aggregate_loader(data_path)

In [ ]:
# plot the averages

# allocate a dict for the holomap
holo_dict = {}

# get the dates for this mouse
mice = data.keys()

# for all the mice
for mouse in mice:
    
    # get the dates for this mouse
    dates = data[mouse].keys()

    # for all the dates
    for day in dates:
        # get the table
        sub_data = data[mouse][day]['average']
#         # wrap the angles in delta heading
#         sub_data['delta_heading'] = fk.wrap(sub_data['delta_heading']+180).copy()
        # get the column labels
        labels = sub_data.columns
        # get the number of rows
        x_axis = sub_data.shape[1]
        # get the array
        sub_array = sub_data.to_numpy()
        sub_array = fm.normalize_matrix(sub_array, axis=0)
        # create the labels
        y_labels = [((idx+0.5)/(len(labels))-0.5, el) for idx, el in enumerate(labels)]
        raw_image = hv.Image(sub_array, label=mouse+'_'+day)
        raw_image.opts(width=1000, height=800, invert_axes=True, invert_yaxis=True, 
                       invert_xaxis=True, cmap='viridis', yticks=y_labels, tools=['hover'])

        holo_dict[(mouse, day)] = raw_image
# holo_image = hv.Layout(plot_list, group='images').cols(1)
holo_image = pn.panel(hv.HoloMap(holo_dict, kdims=['mouse', 'day']), center=True, widget_location='top')

# holo_image = hv.GridSpace(holo_dict, kdims=['mouse', 'day']).opts(plot_size=800)
holo_image


In [ ]:
# Plot particular neurons

# define the target neurons (specify as mouse, date, neuron number from above plot, parameter)
# target_neurons = [
#                     ['MM_191108_a','2019_12_07',4,'cricket_0_mouse_distance'], 
#                     ['MM_191108_a','2019_12_07',8,'cricket_0_mouse_distance'],
#                     ['MM_191108_a','2019_12_11',4,'cricket_0_mouse_distance'],
#                     ['MM_200129_b','2020_03_13',2,'cricket_0_delta_head'],
#                     ['MM_200129_a','2020_03_04',8,'mouse_speed'],
#                  ]
target_neurons = [
    ['DG_210202_a', '2021_03_31', 5, 'cricket_0_mouse_distance'],
    ['DG_210202_a', '2021_03_31', 5, 'cricket_0_visualangle'],
    ['DG_210202_a', '2021_03_31', 11, 'cricket_0_mouse_distance'],
    ['DG_210202_a', '2021_03_31', 11, 'cricket_0_visualangle'],
]
# allocate memory for a layout
layout_list = []
# for all the neurons
for neuron in target_neurons:
        # get the table
        sub_data = data[neuron[0]][neuron[1]]['average']
        sub_sem = data[neuron[0]][neuron[1]]['sem']
        # get the list of present neurons
        neuron_list = np.unique([el[0] for el in sub_data['calcium'].keys()])
        # get the target neuron name
        neuron_name = neuron_list[neuron[2]]
        # get the particular neuron
        max_calcium = np.max(sub_data['calcium'][neuron_name])
        min_calcium = np.min(sub_data['calcium'][neuron_name])
        calcium_data = (sub_data['calcium'][neuron_name]-min_calcium)/(max_calcium-min_calcium)
        calcium_sem = sub_sem['calcium'][neuron_name]/max_calcium
#         calcium_data = fm.normalize_matrix(sub_data['calcium'][neuron_name],axis=0)
#         calcium_sem = fm.normalize_matrix(sub_data['calcium'][neuron_name],axis=0)
        # get the parameter
        max_parameter = np.max(sub_data[neuron[3]][neuron_name])
        min_parameter = np.min(sub_data[neuron[3]][neuron_name])
        param_data = (sub_data[neuron[3]][neuron_name]-min_parameter)/(max_parameter-min_parameter)
        param_sem = sub_sem[neuron[3]][neuron_name]/max_parameter
#         param_data = fm.normalize_matrix(sub_data[neuron[3]][neuron_name],axis=0)
#         param_sem = fm.normalize_matrix(sub_data[neuron[3]][neuron_name],axis=0)
        
        # plot
        # get the x axis vector
        x_vector = np.array(np.arange(calcium_data.shape[0]))
        calcium_curve = hv.Curve((x_vector, calcium_data), 'Time (s)', label='Ca')
        calcium_shade = hv.Spread((x_vector, calcium_data, calcium_sem))
        
        param_curve = hv.Curve((x_vector, param_data), label=neuron[3].replace('_', ' ').replace('distance', 'dist.')+' (px)')
        param_shade = hv.Spread((x_vector, param_data, param_sem))
        
#         plot_first = df_first.hvplot.line(height=800, width=3500, legend=False, value_label="first_y_label").opts(bgcolor="black")
#         plot_second = df_second.hvplot.scatter(height=800, width=3500, legend=False, value_label="second_y_label").opts(bgcolor="black")
#         (plot_first + plot_second).cols(1)
        
        # assemble the overlay
        overlay = calcium_curve*calcium_shade*param_curve*param_shade
#         overlay = calcium_curve*param_curve
        overlay.opts(opts.Curve(width=400, height=400, toolbar=None, 
                        hooks=[fp.margin], fontsize=15, line_width=2, xticks=3, yticks=3),
            opts.Overlay(legend_position='top', text_font='Arial'))

#         # assemble the save path
#         save_path = os.path.join(paths.figures_path,'_'.join((save_name,*[str(el) for el in neuron], suffix,'singleATA_new.png')))
#         hv.save(overlay,save_path)    
    
        # append to the layout
        layout_list.append(overlay)
# show the layout
layout = hv.Layout(layout_list)

layout
#         # wrap the angles in delta heading
#         sub_data['delta_heading'] = fk.wrap(sub_data['delta_heading']+180).copy()
#         # get the column labels
#         labels = sub_data.columns
#         # get the number of rows
#         x_axis = sub_data.shape[1]
#         # get the array
#         sub_array = sub_data.to_numpy()
#         sub_array = fm.normalize_matrix(sub_array, axis=0)

In [ ]:
# plot the spatial "RFs" of each neuron

# allocate a dict for the holomap
holo_dict = {}
holo_dict2 = {}

# get the dates for this mouse
mice = data.keys()
# initialize a dict for the ranges
range_dict = {}

# for all the mice
for mouse in mice:
    
    # get the dates for this mouse
    dates = data[mouse].keys()

    # for all the dates
    for day in dates:
        # get the table
        sub_data = data[mouse][day]['peaks']
        # get a list of the parameters
        column_list = sub_data.columns
        # run through the fields finding the max and min
        for col in column_list:
            # if it's a cell, skip it
            if 'cell' in col:
                continue
            # if it's a random field, skip it
            if 'random' in col:
                continue
            # if it's index or time, skip it
            if col in ['time_vector', 'index', 'peak_id', 'peak_frame']:
                continue
            # check whether the field exists in the dictionary
            if col in range_dict.keys():       
                range_dict[col] = [np.min([range_dict[col][0], np.min(sub_data[col])]), 
                                   np.max([range_dict[col][1], np.max(sub_data[col])])]
            else:
                range_dict[col] = [np.min(sub_data[col]), np.max(sub_data[col])]

        # get the cells
        cells = np.unique(sub_data.cell)
        
        for cell in cells:
            # get the data for this cell
            cell_data = sub_data.loc[sub_data['cell']==cell]
            
            # center the cricket coordinates
#             cell_data['cricket_x'] = cell_data['cricket_x'] - cell_data['mouse_x']
#             cell_data['cricket_y'] = cell_data['cricket_y'] - cell_data['mouse_y']

#             # rotate to the orientation of the mouse
#             rotated_points = fk.rotate_points(cell_data[['cricket_x', 'cricket_y']].to_numpy(), 
#                                               np.expand_dims(cell_data['cricket_heading'].to_numpy(), axis=1))
            
             # get the color range
            c_range = (np.min(cell_data.calcium), np.max(cell_data.calcium))
            
            # create the traces
            trajectory1 = hv.Scatter(cell_data, kdims=['mouse_x'], vdims=['mouse_y', 'calcium'])
            trajectory2 = hv.Scatter(cell_data, kdims=['cricket_0_x'], vdims=['cricket_0_y', 'calcium'])
            trajectory3 = hv.Scatter(([0], [0]))
            trajectory = trajectory1*trajectory2*trajectory3
        
            scatter = hv.Points(cell_data, kdims=['cricket_0_delta_head', 'cricket_0_mouse_distance'], 
                                                        vdims=['cricket_0_mouse_distance', 'calcium'])

            # apply opts
            trajectory1.opts(color='calcium', cmap='viridis', clim=c_range)
            trajectory3.opts(size=5)
            scatter.opts(color='calcium', cmap='viridis', clim=c_range, size=3, 
                         alpha=0.5, xlabel='Head Ori. (deg)', ylabel='mouse cricket distance (px)')
            
            # store for plotting inline
            holo_dict[(mouse,day,cell)] = trajectory
            holo_dict2[(mouse, day, cell)] = scatter

holo_image = pn.panel(hv.HoloMap(holo_dict, kdims=['mouse', 'day', 'cell']).opts(shared_axes=False)+
                      hv.HoloMap(holo_dict2, kdims=['mouse', 'day', 'cell']).opts(shared_axes=False), center=True, 
                      widget_location='top', widgets={'cell': pn.widgets.DiscreteSlider})

holo_image     


In [ ]:
# save particular trajectories and scatters

# plot the spatial "RFs" of each neuron
# define the target neurons (specify as mouse, date, neuron number from above plot, parameter)
target_neurons = [
                    ['MM_191108_a','2019_12_07',4,'cricket_0_delta_head','cricket_0_mouse_distance'], 
                    ['MM_191108_a','2019_12_07',8,'cricket_0_delta_head','cricket_0_mouse_distance'],
                    ['MM_191108_a','2019_12_11',4,'cricket_0_delta_head','cricket_0_mouse_distance'],
                 ]

# allocate memory to store the selected plots for inline
plot_list = []
# for all the target neurons
for neuron in target_neurons:
    
    # get the variables
    mouse = neuron[0]
    day = neuron[1]
    neuron_id = neuron[2]
    para0 = neuron[3]
    para1 = neuron[4]
    
    # get the ranges for the histogram
    x_edges = np.linspace(range_dict[para0][0], range_dict[para0][1], num=13)
#     y_edges = np.linspace(range_dict[para1][0], range_dict[para1][1], num=13)
    y_edges = np.linspace(0, 40, num=13)

    # get the table
    sub_data = data[mouse][day]['peaks']

    # get the cells
    cells = np.unique(sub_data.cell)
    # get the target cell
    target_cell = cells[neuron_id]
        
    # get the data for this cell
    cell_data = sub_data.loc[sub_data['cell']==target_cell]
    print(cell_data)

     # get the color range
    c_range = (np.min(cell_data.calcium), np.max(cell_data.calcium))
    
    
    # get the RF histogram for the cell
    
    # get the raw data from the full data for the null distribution
    whole_day = data_full[mouse][day]

    # get the distribution for the variables of interest
    H_whole = np.histogram2d(whole_day[para0], whole_day[para1],
                                     [x_edges, y_edges], density=False)
    H = np.histogram2d(cell_data[para0], cell_data[para1],
                                 [x_edges, y_edges], density=False)

    # bin it and store
    cell_histogram = np.log((H[0]/cell_data[para0].shape[0])/(H_whole[0]/whole_day[para0].shape[0]))
    cell_histogram[np.isinf(cell_histogram)] = 0
    
    
    
    
#     # log the calcium
#     cell_data['calcium'] = np.log(cell_data['calcium'].copy())

    # create the traces
    trajectory1 = hv.Scatter(cell_data, kdims=['mouse_x'], vdims=['mouse_y', 'calcium'])
#     trajectory2 = hv.Scatter(cell_data, kdims=['cricket_x'], vdims=['cricket_y', 'calcium'])
    
    # plot the map
    image1 = hv.Image(cell_histogram, [el.replace('_', ' ') for el in [para0, para1]],
                     bounds=[x_edges[0], y_edges[0], x_edges[-1], y_edges[-1]])
    image1.opts(width=fp.pix(5), height=fp.pix(4.1), cmap='coolwarm', colorbar=True, xticks=3, yticks=3,
              hooks=[fp.margin], toolbar=None, fontsize=fp.font_sizes['small'], 
               colorbar_opts={'major_label_text_align': 'left'}, clim=(np.min(cell_histogram), np.max(cell_histogram)))

    # assemble the save path
    save_path = os.path.join(paths.figures_path,'_'.join((save_name,*[str(el) for el in neuron], suffix,'histogram.png')))
    hv.save(image1,save_path) 

#     scatter = hv.Points(cell_data, kdims=[para0, para1], 
#                                                 vdims=[para1, 'calcium'])


    # apply opts
    trajectory1.opts(width=fp.pix(5), height=fp.pix(4.1), color='calcium', cmap='viridis', clim=c_range, xlabel='mouse x', 
                     ylabel='mouse y', xticks=0, yticks=0,toolbar=None, 
                        hooks=[fp.margin], line_width=line_width, fontsize=fp.font_sizes['small'], size=20, colorbar=True,
                    colorbar_opts={'major_label_text_align': 'left'})
#     trajectory2.opts(size=20)
#     scatter.opts(width=fp.pix(5), height=fp.pix(4.1),color='calcium', cmap='viridis', clim=c_range, size=20, alpha=0.5,
#                 xticks=3, yticks=3, hooks=[fp.margin], line_width=line_width,toolbar=None, fontsize=fp.font_sizes['small'],
#                 colorbar=True, colorbar_opts={'major_label_text_align': 'left'},
#                 xlabel='Head Ori. (deg)', ylabel='mouse cricket dist. (px)')
    
    trajectory = trajectory1#*trajectory2
#     trajectory.opts(text_font='Arial')
    
    # assemble the save path
    save_path = os.path.join(paths.figures_path,'_'.join((save_name,*[str(el) for el in neuron], suffix,'trajectory.png')))
    hv.save(trajectory,save_path)
    
#     save_path = os.path.join(paths.figures_path,'_'.join((save_name,*[str(el) for el in neuron], suffix,'scatter.png')))
#     hv.save(scatter,save_path)
    
    # store them for plotting inline
    plot_list.append(trajectory)
    plot_list.append(image1)
#     plot_list.append(scatter)

    # create the overlay
#     overlay = trajectory+scatter
#     # run the opts
#     overlay.opts(opts.Scatter(toolbar=None, 
#                 hooks=[fp.margin], line_width=line_width, xticks=3, yticks=3),
#                 opts.Overlay(width=200, height=200, text_font='Arial'))




layout = hv.Layout(plot_list)
layout

In [ ]:
# plot averages per day

# get the dates for this mouse
mice = data.keys()
# define the parameters to bin
# parameter_list = ['delta_head', 'mouse_cricket_distance']
parameter_list = ['cricket_x', 'cricket_y']
# parameter_list = ['mouse_x', 'mouse_y']
# parameter_list = ['mouse_speed', 'cricket_speed']
# parameter_list = ['time_vector', 'mouse_cricket_distance']
# x_edges = np.arange(0, 150,14)
# y_edges = np.arange(0, 150,14)

# define the grid for binning
x_edges = np.linspace(range_dict[parameter_list[0]][0], range_dict[parameter_list[0]][1], num=14)
y_edges = np.linspace(range_dict[parameter_list[1]][0], range_dict[parameter_list[1]][1], num=14)

# allocate memory for the type means
type_means = []
type_images = []
# # for the real and random maps
# for maptype in ['', 'random_']:
# allocate memory for the maps
# animal_maps = []
animal_means = []

# define the parameter names
para0 = parameter_list[0]
para1 = parameter_list[1]

# allocate memory for the whole maps
whole_list = []

# for all the mice
for mouse in mice:
    # allocate memory for the maps
    map_list = []

    # get the dates for this mouse
    dates = data[mouse].keys()

    # for all the dates
    for day in dates:
        # get the raw data from the full data for the null distribution
        whole_day = data_full[mouse][day]
        
        # get the distribution for the variables of interest
        H_whole = np.histogram2d(whole_day[para0], whole_day[para1],
                                         [x_edges, y_edges], density=False)
        
#         # get the time
#         H_whole[0] = H_whole[0]/(whole_day[para0].shape[0]/10)
        # get the probability
        whole_probability = H_whole[0]/(whole_day[para0].shape[0])
#         print(H_time)
        # save the map
        whole_list.append(whole_probability)
        
        # get the table
        sub_data = data[mouse][day]['peaks']
        # get the cells
        cells = np.unique(sub_data.cell)
        for cell in cells:
            # get the data for this cell
            cell_data = sub_data.loc[sub_data['cell']==cell, :]
            H = np.histogram2d(cell_data[para0], cell_data[para1],
                                         [x_edges, y_edges], density=False)
            
            # get the time
            H_time = cell_data[para0].shape[0]
            
            # bin it and store
            map_list.append(H[0]/(whole_probability*H_time))


    # plot the average histogram for the animal
    mean_map = np.nanmean(map_list, axis=0)

    # remove the nans
    mean_map[np.isnan(mean_map)] = 0
    mean_map[np.isinf(mean_map)] = 0
    mean_map = np.flipud(mean_map.T)
    # store for corrected map calculation
    animal_means.append(mean_map)

# calculate the average to plot
activity_average = np.log(np.nanmean(animal_means, axis=0))
activity_average[np.isinf(activity_average)] = 0

# define the clim
# clim = (np.nanmin(activity_average), np.nanmax(activity_average))
clim = (-1.5, 1.5)
# plot the map
image1 = hv.Image(activity_average, [el.replace('_', ' ') for el in parameter_list],
                 bounds=[x_edges[0], y_edges[0], x_edges[-1], y_edges[-1]])
image1.opts(width=fp.pix(5), height=fp.pix(4.1), cmap='coolwarm', colorbar=True, xticks=3, yticks=3,
          hooks=[fp.margin], toolbar=None, fontsize=fp.font_sizes['small'], 
           colorbar_opts={'major_label_text_align': 'left'}, clim=clim)

# assemble the save path
save_path = os.path.join(paths.figures_path,'_'.join((save_name, *parameter_list, suffix, 'histogram2.png')))
hv.save(image1,save_path)


# plot the map
whole_map = np.log(np.flipud(np.nanmean(whole_list, axis=0).T))
whole_map[np.isinf(whole_map)] = 0

image2 = hv.Image(whole_map, [el.replace('_', ' ') for el in parameter_list],
                 bounds=[x_edges[0], y_edges[0], x_edges[-1], y_edges[-1]])
image2.opts(width=fp.pix(5), height=fp.pix(4.1), cmap='reds', colorbar=True, xticks=3, yticks=3,
          hooks=[fp.margin], toolbar=None, fontsize=fp.font_sizes['small'], 
           colorbar_opts={'major_label_text_align': 'left'}, clim=(np.min(whole_map), np.max(whole_map)))

# assemble the save path
save_path = os.path.join(paths.figures_path,'_'.join((save_name, *parameter_list, suffix, 'base_histogram2.png')))
hv.save(image2,save_path)

layout = image1+image2
layout.opts(opts.Image(tools=['hover']))
layout
